# Promethium Quickstart: Basic Usage

This notebook provides a minimal end-to-end example of using the Promethium library for seismic data recovery.

**Prerequisites:**
- Python 3.10+
- No prior Promethium experience required

**Estimated Runtime:** 2-3 minutes

**Library Version:** promethium-seismic==1.0.0

## 1. Installation and Environment Setup

If running in a fresh environment (Kaggle, Colab, new local venv), uncomment and run the installation cell.

In [4]:
# Uncomment to install from PyPI:
# !pip install promethium-seismic==1.0.0

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import and verify version
import promethium

print(f"Promethium version: {promethium.__version__}")
assert promethium.__version__ == "1.0.0", "Version mismatch - please install promethium-seismic==1.0.0"

ModuleNotFoundError: No module named 'promethium'

In [ ]:
# Import required functions and classes
from promethium import (
    SeismicRecoveryPipeline,
    evaluate_reconstruction,
    generate_synthetic_traces,
    add_noise,
    set_seed,
    get_device,
    plot_comparison,
)

import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
set_seed(42)

# Detect available device
device = get_device()
print(f"Using device: {device}")

## 2. Generate Synthetic Dataset

For this quickstart, we generate a small synthetic seismic dataset with added noise.
In practice, you would load real data using `promethium.load_segy()` or similar functions.

In [ ]:
# Generate clean synthetic traces
clean_data, metadata = generate_synthetic_traces(
    n_traces=50,
    n_samples=500,
    sample_rate=250.0,
    seed=42
)

print(f"Generated {metadata['n_traces']} traces")
print(f"Samples per trace: {metadata['n_samples']}")
print(f"Sample rate: {metadata['sample_rate']} Hz")
print(f"Duration: {metadata['duration']:.2f} seconds")
print(f"Data shape: {clean_data.shape}")

In [ ]:
# Add noise to simulate corrupted data
noisy_data = add_noise(clean_data, noise_level=0.3, seed=42)

# Visualize a single trace before and after noise
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

t = np.arange(metadata['n_samples']) / metadata['sample_rate']

axes[0].plot(t, clean_data[0], 'b-', linewidth=0.8)
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Clean Trace')
axes[0].grid(True, alpha=0.3)

axes[1].plot(t, noisy_data[0], 'r-', linewidth=0.8)
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Amplitude')
axes[1].set_title('Noisy Trace (30% noise level)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Create Recovery Pipeline

Promethium provides pre-configured pipelines for common recovery tasks.
Here we use the `unet_denoise_v1` preset for denoising.

In [ ]:
# List available presets
presets = SeismicRecoveryPipeline.list_presets()
print("Available pipeline presets:")
for preset in presets:
    print(f"  - {preset}")

In [ ]:
# Create pipeline from preset
pipeline = SeismicRecoveryPipeline.from_preset("unet_denoise_v1")

print(f"Pipeline model: {pipeline.model_name}")
print(f"Preprocessing steps: {pipeline.preprocessing}")
print(f"Postprocessing steps: {pipeline.postprocessing}")

## 4. Run Recovery

Execute the pipeline on the noisy data to obtain reconstructed (denoised) traces.

In [ ]:
# Run the recovery pipeline
# Note: For this demo, we simulate the inference since no trained model is loaded
# In production, the pipeline would use actual model weights

try:
    reconstructed_data = pipeline.run(noisy_data)
except Exception as e:
    print(f"Note: Full inference requires trained model weights.")
    print(f"For demo purposes, using simple denoising filter.")
    # Fallback: simple smoothing filter for demonstration
    from scipy.ndimage import gaussian_filter1d
    reconstructed_data = np.array([gaussian_filter1d(trace, sigma=2) for trace in noisy_data])

print(f"Reconstructed data shape: {reconstructed_data.shape}")

## 5. Evaluate Reconstruction Quality

Compute standard metrics to assess reconstruction quality.

In [ ]:
# Compute all evaluation metrics
metrics = evaluate_reconstruction(clean_data, reconstructed_data)

print("Reconstruction Quality Metrics:")
print("-" * 40)
for name, value in metrics.items():
    if name in ['snr', 'psnr']:
        print(f"{name.upper():>20}: {value:>10.2f} dB")
    else:
        print(f"{name:>20}: {value:>10.4f}")

In [ ]:
# Compare noisy input vs reconstructed output
noisy_metrics = evaluate_reconstruction(clean_data, noisy_data)

print("\nComparison: Noisy vs Reconstructed")
print("-" * 50)
print(f"{'Metric':>20} {'Noisy':>12} {'Reconstructed':>15}")
print("-" * 50)
for name in metrics.keys():
    print(f"{name:>20} {noisy_metrics[name]:>12.4f} {metrics[name]:>15.4f}")

## 6. Visualize Results

Compare original, noisy, and reconstructed traces visually.

In [ ]:
# Plot comparison for a single trace
trace_idx = 10
t = np.arange(metadata['n_samples']) / metadata['sample_rate']

fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)

# Original
axes[0].plot(t, clean_data[trace_idx], 'b-', linewidth=0.8)
axes[0].set_ylabel('Amplitude')
axes[0].set_title(f'Original Clean Trace (Index {trace_idx})')
axes[0].grid(True, alpha=0.3)

# Noisy
axes[1].plot(t, noisy_data[trace_idx], 'r-', linewidth=0.8)
axes[1].set_ylabel('Amplitude')
axes[1].set_title('Noisy Input')
axes[1].grid(True, alpha=0.3)

# Reconstructed
axes[2].plot(t, reconstructed_data[trace_idx], 'g-', linewidth=0.8)
axes[2].set_ylabel('Amplitude')
axes[2].set_title('Reconstructed Output')
axes[2].grid(True, alpha=0.3)

# Overlay comparison
axes[3].plot(t, clean_data[trace_idx], 'b-', linewidth=0.8, label='Original', alpha=0.7)
axes[3].plot(t, reconstructed_data[trace_idx], 'g--', linewidth=0.8, label='Reconstructed', alpha=0.7)
axes[3].set_xlabel('Time (s)')
axes[3].set_ylabel('Amplitude')
axes[3].set_title('Overlay: Original vs Reconstructed')
axes[3].legend(loc='upper right')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Summary

In this quickstart notebook, we demonstrated:

1. **Installation**: `pip install promethium-seismic==1.0.0`
2. **Data Generation**: Creating synthetic seismic traces with noise
3. **Pipeline Setup**: Using `SeismicRecoveryPipeline.from_preset()`
4. **Recovery**: Running the pipeline on noisy data
5. **Evaluation**: Computing SNR, MSE, PSNR, SSIM, and other metrics
6. **Visualization**: Comparing original, noisy, and reconstructed traces

### Next Steps

- **02_data_ingestion_and_quality_control.ipynb**: Load real SEG-Y data
- **03_signal_processing_basics.ipynb**: Apply filters and transforms
- **05_deep_learning_unet_reconstruction.ipynb**: Deep learning inference
- **08_kaggle_and_colab_integration.ipynb**: Cloud environment usage